In [0]:
%scala
// val sparkConf = new SparkConf().setMaster("local[*]").setAppName("spark")
// val sparkContext = new SparkContext(sparkConf)
val rdd1 = spark.parallelize(List(1,2,3,4))
val rdd2 = spark.makeRDD(List(1,2,3,4))
rdd1.collect().foreach(println)
rdd2.collect().foreach(println)

In [0]:
# File location and type
file_location = "/FileStore/tables/2014_summary.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-802265700720709>:15
      8 delimiter = ","
     10 # The applied options are for CSV files. For other file types, these will be ignored.
     11 df = spark.read.format(file_type) \
     12   .option("inferSchema", infer_schema) \
     13   .option("header", first_row_is_header) \
     14   .option("sep", delimiter) \
---> 15   .load(file_location)
     17 display(df)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:307, in DataFrameReader.load(self

In [0]:
test = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/garyyihengsong@gmail.com/test.csv")
train = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/garyyihengsong@gmail.com/train.csv")

In [0]:
train = train.na.fill(-1)
test = test.na.fill(-1)
train = train.na.fill('-1')
test = test.na.fill('-1')

In [0]:
test.filter(test.Product_Category_2.isNull()).show()

+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+
|User_ID|Product_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+



In [0]:

test.select('Product_ID').distinct().count()

3491

In [0]:
train.select('Product_ID').distinct().count()

3631

In [0]:
common = train.select('Product_ID').intersect(test.select('Product_ID'))

In [0]:
common.show()

+----------+
|Product_ID|
+----------+
| P00014542|
| P00251242|
|  P0096642|
| P00087842|
| P00220442|
| P00069042|
| P00184942|
| P00285442|
| P00190242|
| P00145042|
| P00193542|
| P00248942|
| P00058442|
| P00085442|
| P00274942|
| P00231342|
| P00346142|
| P00031342|
|  P0097242|
| P00036842|
+----------+
only showing top 20 rows



In [0]:
train = common.join(train, ['Product_ID'], 'inner')
test = common.join(test, ['Product_ID'], 'inner')


In [0]:
train.select('Product_ID').distinct().count()

3445

In [0]:
common.select('Product_ID').distinct().count()

3445

In [0]:
test.select('Product_ID').distinct().count()

3445

In [0]:
diff_cat_in_train_test=test.select('Product_ID').subtract(train.select('Product_ID'))

diff_cat_in_train_test.distinct().count()

0

In [0]:
train.createOrReplaceTempView('df1')
t = spark.sql('select * from df1 where Product_ID is null')
t.show()

+----------+-------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|Product_ID|User_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+----------+-------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
+----------+-------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+



In [0]:
from pyspark.ml.feature import StringIndexer
plan_indexer = StringIndexer(inputCol = 'Product_ID', outputCol = 'product_id_trans')
labeller = plan_indexer.setHandleInvalid("skip").fit(train)

In [0]:
Train1 = labeller.transform(train)
Test1 = labeller.transform(test)
Train1.show(2)

+----------+-------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+----------------+
|Product_ID|User_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|product_id_trans|
+----------+-------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+----------------+
| P00069042|1000001|     F|0-17|        10|            A|                         2|             0|                 3|                -1|                -1|    8370|           760.0|
| P00248942|1000001|     F|0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|           178.0|
+----------+-------+------+----+----------+-------------+--------------------------+-

In [0]:
Train1.createOrReplaceTempView('df2')
t = spark.sql('select * from df2 where product_id_trans is null')
t.show()

+----------+-------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+----------------+
|Product_ID|User_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|product_id_trans|
+----------+-------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+----------------+
+----------+-------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+----------------+



In [0]:
Train1.printSchema()

root
 |-- Product_ID: string (nullable = false)
 |-- User_ID: string (nullable = false)
 |-- Gender: string (nullable = false)
 |-- Age: string (nullable = false)
 |-- Occupation: string (nullable = false)
 |-- City_Category: string (nullable = false)
 |-- Stay_In_Current_City_Years: string (nullable = false)
 |-- Marital_Status: string (nullable = false)
 |-- Product_Category_1: string (nullable = false)
 |-- Product_Category_2: string (nullable = false)
 |-- Product_Category_3: string (nullable = false)
 |-- Purchase: string (nullable = false)
 |-- product_id_trans: double (nullable = false)



In [0]:
from pyspark.ml.feature import RFormula
formula = RFormula(formula="Purchase ~ Age+ Occupation +City_Category+Stay_In_Current_City_Years+Product_Category_1+Product_Category_2+ Gender",
                   featuresCol="features",labelCol="label")

In [0]:
t1 = formula.fit(Train1)
train1 = t1.transform(Train1)
test1 = t1.transform(Test1)


In [0]:
test1.show()

+----------+-------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+----------------+--------------------+
|Product_ID|User_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|product_id_trans|            features|
+----------+-------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+----------------+--------------------+
| P00128942|1000004|     M|46-50|         7|            B|                         2|             1|                 1|                11|                -1|            47.0|(67,[3,8,26,29,33...|
| P00113442|1000009|     M|26-35|        17|            C|                         0|             0|                 3|                 5|                -1|           678.0|(67,[0,10,27,38,5...|
| P00288442|1000010|

In [0]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor()
(train_cv, test_cv) = train1.randomSplit([0.7, 0.3])
model1 = rf.fit(train_cv)
predictions = model1.transform(test_cv)
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator()
mse = evaluator.evaluate(predictions,{evaluator.metricName:"mse" })
import numpy as np
np.sqrt(mse), mse
model = rf.fit(train1)
predictions1 = model.transform(test1)
df = predictions1.selectExpr("User_ID as User_ID", "Product_ID as Product_ID", 'prediction as Purchase')
df.show()

+-------+----------+------------------+
|User_ID|Product_ID|          Purchase|
+-------+----------+------------------+
|1000004| P00128942| 4346.123915815542|
|1000009| P00113442|5908.0231635274895|
|1000010| P00288442|1466.9371923284875|
|1000010| P00145342| 4867.982987395578|
|1000011| P00053842| 4867.982987395578|
|1000013| P00350442| 5743.378871099556|
|1000013| P00155442| 4346.123915815542|
|1000013|  P0094542| 6328.447859309751|
|1000015| P00161842| 9250.863025800794|
|1000022| P00067942|1505.2082037643263|
|1000026| P00046742| 4152.243151463894|
|1000026| P00040042|1550.0871410861919|
|1000026| P00196542|1371.3283325353154|
|1000026| P00004542|1371.3283325353154|
|1000028| P00159542| 9377.028636869385|
|1000029| P00111542| 5590.689148901201|
|1000033| P00121042| 8692.820936568187|
|1000033| P00344442|1368.7862300323252|
|1000034| P00265242|1419.7896494592208|
|1000035|  P0096642| 5743.378871099556|
+-------+----------+------------------+
only showing top 20 rows



In [0]:
predictions1.show(5)

+----------+-------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+----------------+--------------------+------------------+
|Product_ID|User_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|product_id_trans|            features|        prediction|
+----------+-------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+----------------+--------------------+------------------+
| P00128942|1000004|     M|46-50|         7|            B|                         2|             1|                 1|                11|                -1|            47.0|(67,[3,8,26,29,33...| 4346.123915815542|
| P00113442|1000009|     M|26-35|        17|            C|                         0|             0|                 3|                 5|  